In [25]:
import gtsam
import numpy as np

In [67]:
# sensor model p(X|Z)
classes = ['false_det',
           'movable_object.barrier',
           'vehicle.bicycle',
           'vehicle.bus.bendy',
           'vehicle.bus.rigid',
           'car',
           'vehicle.construction',
           'vehicle.motorcycle',
           'human.pedestrian.adult',
           'human.pedestrian.child',
           'human.pedestrian.construction_worker',
           'human.pedestrian.police_officer',
           'movable_object.trafficcone',
           'vehicle.trailer',
           'vehicle.truck']

# detection
detections = ['missed_det',
              'barrier',
              'bicycle',
              'bus',
              'car',
              'motorcycle',
              'construction_vehicle',
              'motorcycle',
              'pedestrian',
              'traffic_cone',
              'trailer',
              'truck']

test_classes = ['nothing',
                'person',
                'furniture']
test_detections = ['no_leg',
                   'leg']

In [68]:
def compute_matrix_spec(array):
    spec = ''
    
    for row in range(array.shape[0]):
        if row!=0:
            spec+=' '    
        for col in range(array.shape[1]):
            if col!=0:
                spec+='/'
            spec+=str(array[row,col])
    
    return spec

# Test

In [88]:
cat_idx = 10
det_idx = 20

test_cond_matrix = np.array([[.9, .1],[.2, .8],[.4, .6]])
print(test_cond_matrix)

spec_str = compute_matrix_spec(test_cond_matrix)
print(spec_str)

[[0.9 0.1]
 [0.2 0.8]
 [0.4 0.6]]
0.9/0.1 0.2/0.8 0.4/0.6


In [89]:
# Compute prior
n_test_cats = len(test_classes)
n_test_dets = len(test_detections)
category_prior = gtsam.DiscreteDistribution((cat_idx,n_test_cats),[1/n_test_cats]*n_test_cats)
print(category_prior)

Discrete Prior
 P( 10 ):
 Leaf 0.33333333




In [90]:
# Sensor model
p_Z_X = gtsam.DiscreteConditional((det_idx,n_test_dets),[(cat_idx,n_test_cats)],spec_str)
print(p_Z_X)

# Detection indices
no_leg_det_idx = 0
leg_det_idx = 1 

Discrete Conditional
 P( 20 | 10 ):
 Choice(20) 
 0 Choice(10) 
 0 0 Leaf  0.9
 0 1 Leaf  0.2
 0 2 Leaf  0.4
 1 Choice(10) 
 1 0 Leaf  0.1
 1 1 Leaf  0.8
 1 2 Leaf  0.6




In [101]:
no_leg_detected_factor = p_Z_X.likelihood(no_leg_det_idx)
print(no_leg_detected_factor)

leg_detected_factor = p_Z_X.likelihood(leg_det_idx)
print(leg_detected_factor)

DecisionTreeFactor
 f[ (10,3), ]
 Choice(10) 
 0 Leaf  0.9
 1 Leaf  0.2
 2 Leaf  0.4

DecisionTreeFactor
 f[ (10,3), ]
 Choice(10) 
 0 Leaf  0.1
 1 Leaf  0.8
 2 Leaf  0.6



In [132]:
norm_posterior = leg_detected_factor

In [175]:
meas_idx = 1 # 0 for no leg, 1 for leg detected
unnorm_posterior = p_Z_X.likelihood(meas_idx)*norm_posterior
norm_posterior = gtsam.DiscreteDistribution(unnorm_posterior)
print(norm_posterior)
print()
print(test_classes[norm_posterior.argmax()])
print(norm_posterior(norm_posterior.argmax()))

Discrete Prior
 P( 10 ):
 Choice(10) 
 0 Leaf 1.604018e-22
 1 Leaf 0.82937544
 2 Leaf 0.17062456



person
0.8293754377177844


In [142]:
help(gtsam.DiscreteDistribution)

Help on class DiscreteDistribution in module gtsam.gtsam:

class DiscreteDistribution(DiscreteConditional)
 |  Method resolution order:
 |      DiscreteDistribution
 |      DiscreteConditional
 |      DecisionTreeFactor
 |      DiscreteFactor
 |      Factor
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(...)
 |      __call__(self: gtsam.gtsam.DiscreteDistribution, arg0: int) -> float
 |  
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: gtsam.gtsam.DiscreteDistribution) -> None
 |      
 |      2. __init__(self: gtsam.gtsam.DiscreteDistribution, f: gtsam.gtsam.DecisionTreeFactor) -> None
 |      
 |      3. __init__(self: gtsam.gtsam.DiscreteDistribution, key: Tuple[int, int], spec: str) -> None
 |      
 |      4. __init__(self: gtsam.gtsam.DiscreteDistribution, key: Tuple[int, int], spec: List[float]) -> None
 |  
 |  __repr__(...)
 |      __repr__(se

# The real thing

In [18]:
n_categories = len(classes)
category_prior = gtsam.DiscreteDistribution((0,n_categories),[1/n_categories]*n_categories)
print(category_prior)

Discrete Prior
 P( 0 ):
 Leaf 0.066666667




In [23]:
# there are n_category rows, with n_detections columns
# each row represents the likelihood that the category will yield that detection class, and each row sums to 1
p_Z_X = gtsam.DiscreteConditional((1,n_detections),[(0,n_categories)],
                                 "1/1/1/1/1/1/1/1/1/1/1/1 2/8/0/0/0/0/0/0/0/0/0/0 ")
help(gtsam.DiscreteConditional)

Help on class DiscreteConditional in module gtsam.gtsam:

class DiscreteConditional(DecisionTreeFactor)
 |  Method resolution order:
 |      DiscreteConditional
 |      DecisionTreeFactor
 |      DiscreteFactor
 |      Factor
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: gtsam.gtsam.DiscreteConditional) -> None
 |      
 |      2. __init__(self: gtsam.gtsam.DiscreteConditional, nFrontals: int, f: gtsam.gtsam.DecisionTreeFactor) -> None
 |      
 |      3. __init__(self: gtsam.gtsam.DiscreteConditional, key: Tuple[int, int], spec: str) -> None
 |      
 |      4. __init__(self: gtsam.gtsam.DiscreteConditional, key: Tuple[int, int], parents: gtsam.gtsam.DiscreteKeys, spec: str) -> None
 |      
 |      5. __init__(self: gtsam.gtsam.DiscreteConditional, key: Tuple[int, int], parents: List[Tuple[int, int]], spec: str) ->